In [7]:
from __future__ import division
import matplotlib.pyplot as plt
import numpy as np
import math
import copy
import tsplib95

In [8]:
problem = tsplib95.load('../data/domain/bays29.tsp')

In [3]:
peso = np.copy(problem.edge_weights)
masked = np.ma.masked_equal(peso, 0, copy=False)
masked = np.ma.masked_equal(peso, 0, copy=False)
problem.edge_weights[16][21] = 45
problem.dimension

29

In [4]:
#calculate distance
list(problem.get_nodes())
edge1 = 6
edge2 = 7
dist = np.min(problem.get_weight(edge1,edge2))
dist
problem.edge_weights

[[0,
  107,
  241,
  190,
  124,
  80,
  316,
  76,
  152,
  157,
  283,
  133,
  113,
  297,
  228,
  129,
  348,
  276,
  188,
  150,
  65,
  341,
  184,
  67,
  221,
  169,
  108,
  45,
  167],
 [107,
  0,
  148,
  137,
  88,
  127,
  336,
  183,
  134,
  95,
  254,
  180,
  101,
  234,
  175,
  176,
  265,
  199,
  182,
  67,
  42,
  278,
  271,
  146,
  251,
  105,
  191,
  139,
  79],
 [241,
  148,
  0,
  374,
  171,
  259,
  509,
  317,
  217,
  232,
  491,
  312,
  280,
  391,
  412,
  349,
  422,
  356,
  355,
  204,
  182,
  435,
  417,
  292,
  424,
  116,
  337,
  273,
  77],
 [190,
  137,
  374,
  0,
  202,
  234,
  222,
  192,
  248,
  42,
  117,
  287,
  79,
  107,
  38,
  121,
  152,
  86,
  68,
  70,
  137,
  151,
  239,
  135,
  137,
  242,
  165,
  228,
  205],
 [124,
  88,
  171,
  202,
  0,
  61,
  392,
  202,
  46,
  160,
  319,
  112,
  163,
  322,
  240,
  232,
  314,
  287,
  238,
  155,
  65,
  366,
  300,
  175,
  307,
  57,
  220,
  121,
  97],
 [80,
  127,


### I discover that I can make a deepcopy of the array, but just to remember I'll let this file here.

In [5]:
weights = np.copy(problem.edge_weights)
weight_copy = copy.deepcopy(weights)

### Testing array slices

In [6]:
arr = np.array([[1, 2, 3, 4, 5], [6, 7, 8, 9, 10]])
arr[1, :]

array([ 6,  7,  8,  9, 10])

### Deep copy again

In [7]:
pop = np.zeros((29,29))
pop = np.random.randint(0, 1+1, size=(29,29))
fit = copy.deepcopy(pop)
print("First -> ",pop[5][5],"\n")
print("Copy -> ",fit[5][5],"\n")
pop[5][5] = 5
print("First -> ",pop[5][5],"\n")
print("Copy -> ",fit[5][5],"\n")

First ->  1 

Copy ->  1 

First ->  5 

Copy ->  1 



In [8]:
pop = np.zeros((29,29))
pop = np.random.randint(0, 1+1, size=(29,29))
fit = np.copy(pop)
print("First -> ",pop[5][5],"\n")
print("Copy -> ",fit[5][5],"\n")
pop[5][5] = 5
print("First -> ",pop[5][5],"\n")
print("Copy -> ",fit[5][5],"\n")

First ->  0 

Copy ->  0 

First ->  5 

Copy ->  0 



### Made a generation of at least one sheep for city (line)

In [9]:
# he first city is 0
# Need to put at least one sheep per city

# Create a random number each iteration and after that made a line of the matrix receive that number
def generate_city(n: int, dim: int) -> list:
    city_matrix = np.zeros((n, dim))
    for i in range(n):
        sheep = np.random.randint(0, dim)
        for j in range(dim):
            if (sheep == j):
                city_matrix[i][j] = 1
    
    return city_matrix

def print_position_of_sheep(city_matrix: list):
    for i in range(len(city_matrix)):
        for j in range(len(city_matrix)):
            if (city_matrix[i][j] != 0):
                print("Posição -> ",j,"\n") 

In [12]:
# Create a function to verify if a certain route is valid
# Need to think in situations where the route is not valid

def verify_route(city_matrix: list) -> str:
    # Append each position in a list, after that verify if the route is valid in a different ways
    valid = "The route is valid!!!"
    route = []
    route.append(0)
    for i in range(len(city_matrix)):
        for j in range(len(city_matrix)):
            if (city_matrix[i][j] == 1):
                route.append(j)
                if (i == (len(city_matrix) - 1) and j != 0):
                    valid = "The route is not valid!!!"
    print("Rota -> ",route,"\n")

    return valid

In [13]:
#print_position_of_sheep(generate_city(30, 30))
n = 30
dim = 30
print(verify_route(generate_city(n, dim)))

Rota ->  [0, 14, 5, 17, 25, 28, 12, 2, 16, 3, 19, 16, 24, 11, 6, 4, 0, 4, 24, 18, 23, 23, 25, 26, 20, 14, 27, 11, 25, 17, 24] 

The route is not valid!!!
